In [1]:
import sklearn
import nltk

from generateWordFrequency import *
from naiveBayes import *
from fileWriteFunctions import *

In [2]:
AllClasses = ['story', 'ask_hn', 'show_hn', 'poll']
delta = 0.5
appendClassPrefix = 'prob_'

In [3]:
unfilteredTrainData = getDataframe(2018)
unfilteredTrainData = addTokenizedColumnofTitle(unfilteredTrainData)

In [4]:
unfilteredTestData = getDataframe(2019)
unfilteredTestData = addTokenizedColumnofTitle(unfilteredTestData)
unfilteredTestData = unfilteredTestData.reset_index()

In [5]:
trainData = unfilteredTrainData.copy()
testData = unfilteredTestData.copy()

trainWords = getWordFrequencyDataframe(trainData,AllClasses)
priorProbabilities = getPriorProbabilities(trainData) 

trainWords = getWordFrequencyDataframe(trainData,AllClasses)
trainWords = obtainDataframeWithClassProbabilities(trainWords, AllClasses, delta, appendClassPrefix)
writeModel(trainWords,'model-2018.txt',AllClasses,appendClassPrefix)
trainWords = renameModelRows(trainWords, AllClasses, appendClassPrefix)
model = [trainWords, priorProbabilities]

testData = generateCondClassProb(testData, model)
testData['predicted'] = generatePrediction(testData,AllClasses)
testResults = comparePredictions(testData,AllClasses)
writeDataframe(testResults,'baseline-result.txt')
check = testResults.comparision.value_counts()
accuracy = check[True]/(check[True]+check[False])

In [6]:
print(accuracy)
df_confusion = pd.crosstab(testResults['Post Type'], testResults['predicted'])
print(df_confusion)

0.9826112305505957
predicted  ask_hn  show_hn   story
Post Type                         
ask_hn       5404        8      42
poll            2        0       4
show_hn       102     4641     160
story         932     1136  124784


In [9]:
df_confusion = pd.crosstab(testResults['Post Type'], testResults['predicted'])
df_confusion['poll'] = 0
df_confusion['precision'] = 0.0
df_confusion['recall'] = 0.0
#df['F1_score'] = 0.0
transpose_matrix = df_confusion.transpose()
for i in transpose_matrix.columns:
    if np.sum(df_confusion[i])==0:
        total = 1
    else:
        total = np.sum(df_confusion[i])
    df_confusion['precision'][i] = df_confusion[i][i]/total
    df_confusion['recall'][i] = df_confusion[i][i]/np.sum(transpose_matrix[i])
df_confusion['F1_score'] = 2/(1/df_confusion['precision']+1/df_confusion['recall'])
print(df_confusion)

predicted  ask_hn  show_hn   story  poll  precision    recall  F1_score
Post Type                                                              
ask_hn       5404        8      42     0   0.839130  0.990832  0.908693
poll            2        0       4     0   0.000000  0.000000  0.000000
show_hn       102     4641     160     0   0.802247  0.946563  0.868451
story         932     1136  124784     0   0.998352  0.983698  0.990971


/home/paras/anaconda3/envs/cu/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/paras/anaconda3/envs/cu/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
